This notebook demonstrates the mechanics of translating an HGVS expression to a VR representation for educational purposes. Users who wish to translate HGVS or other expressions routinely should use ga4gh.vrs.extras.translator.

In [14]:
from ga4gh.core import ga4gh_identify
from ga4gh.vrs import models
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy
from ga4gh.vrs.extras.translator import Translator
from IPython.display import HTML, display

## Translate an expression manually

First, we'll translate NC_000013.11:g.32936732G>C to VR manually to see the evolution of the process. 

In [2]:
# We'll translate this expression to VR:
hgvs_expr = ""

In [3]:
allele = models.Allele(
    location = models.SequenceLocation(
        sequence_id = "refseq:NC_000013.11",
        interval = models.SimpleInterval(
            start = 32936731,
            end = 32936732
        )
    ),
    state = models.SequenceState(
        sequence = "C"
    )
)

allele.as_dict()

{'type': 'Allele',
 'location': {'type': 'SequenceLocation',
  'sequence_id': 'refseq:NC_000013.11',
  'interval': {'type': 'SimpleInterval', 'start': 32936731, 'end': 32936732}},
 'state': {'type': 'SequenceState', 'sequence': 'C'}}

<div class="alert alert-success">
    👍 The above message is a valid VR message. VR requires that
    sequence identifiers use CURIE syntax with a namespace from identifiers.org and
    that locations are specified with interbase coordinates. Using sequence digests
    is NOT required.
    </div>

## Replace the RefSeq sequence with a GA4GH sequence id

In order to use the computed identifier mechanism in VR, the sequence_id MUST use
GA4GH computed sequence identifiers. 

Implementations choose how to provide sequence and sequence accession services

The following uses the [seqrepo REST interface](https://github.com/biocommons/seqrepo-rest-service/). See the vr-python docker instructions for a simple way to set this up.


In [4]:
seqrepo_rest_service_url = "http://localhost:5000/seqrepo"
dp = SeqRepoRESTDataProxy(base_url=seqrepo_rest_service_url)

# In general, one identifier may be related to many others in another namespace
# Therefore, translate_sequence_identifier() returns a list.
# Because there will be only 1 ga4gh sequence digest, we choose the first
# and then replace the sequence id in allele.location.

refseq_ir = str(allele.location.sequence_id)
ga4gh_ir = dp.translate_sequence_identifier(refseq_ir, "ga4gh")[0]
ga4gh_ir

'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT'

In [5]:
# Now, simply replace the identifier with the GA4GH identifier
allele.location.sequence_id = ga4gh_ir
allele.as_dict()

{'type': 'Allele',
 'location': {'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'interval': {'type': 'SimpleInterval', 'start': 32936731, 'end': 32936732}},
 'state': {'type': 'SequenceState', 'sequence': 'C'}}

## Computed Identifiers (optional)

ga4gh_identify() serializes the object and computes the identifier
(See ga4gh_serialize and ga4gh_digest for details)


In [6]:
allele._id = ga4gh_identify(allele)
allele.as_dict()

{'_id': 'ga4gh:VA.n9ax-9x6gOC0OEt73VMYqCBfqfxG1XUH',
 'type': 'Allele',
 'location': {'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'interval': {'type': 'SimpleInterval', 'start': 32936731, 'end': 32936732}},
 'state': {'type': 'SequenceState', 'sequence': 'C'}}

# Using ga4gh.vrs.extras.translator

The VR Translator imports HGVS, SPDI, Beacon, and VCF formats, and appropriate handles more complex cases than shown above.

By default, the translator translates HGVS reference sequences to GA4GH sequence digest identifiers and adds identifiers to the resulting Allele objects.

In [7]:
tlr = Translator(data_proxy=dp)

### HGVS → VR

In [8]:
hgvs_expr1 = "NC_000013.11:g.32936732C>G"
allele1 = tlr.translate_from(hgvs_expr1,'hgvs')
allele1.as_dict()

{'_id': 'ga4gh:VA.gvCtR5KLdng5G31DwajXiH6S3Gjhm5fh',
 'type': 'Allele',
 'location': {'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'interval': {'type': 'SimpleInterval', 'start': 32936731, 'end': 32936732}},
 'state': {'type': 'SequenceState', 'sequence': 'G'}}

### VR → HGVS

Because a GA4GH sequence identifier may have many aliases, each VR Allele
may be expressed as multiple HGVS expressions. For this reason, `translate_to(allele, "hgvs")` returns
a *list* of HGVS expressions, optionally limited to aliases from a specified namespace.

In [9]:
tlr.translate_to(allele, "hgvs")

['NC_000013.11:g.32936732=']

In [10]:
# Most commonly, we'll want expressions from a well-known authority like RefSeq
# Again, there might in general be multiple `refseq` expressions
tlr._to_hgvs(allele, "refseq")

['NC_000013.11:g.32936732=']

In [11]:
# GRC namespaces is handled as a special case: Because aliases are shared 
# between GRCh releases, they're shown only on request
tlr._to_hgvs(allele, "GRCh38")

['13:g.32936732=', 'chr13:g.32936732=']

In [12]:
import json
import yaml
def filter_dict(d):
    try:
        return {k: filter_dict(d[k])
                for k in d
                if not k.startswith("_")}
    except:
        return d
def as_str(s):
    return s if isinstance(s, str) else s.decode()
def dj(o):
    """print VR object as pretty formated json"""
    print(json.dumps(filter_dict(o.as_dict()), indent=2, sort_keys=True))
def dy(fns, o):
    """execute function f in fns on o, returning a yaml block representing the test"""
    r = {
        "in": o.as_dict(),
        "out": {f.__name__: as_str(f(o)) for f in fns}
    }
    print(yaml.dump(filter_dict({o.type._value: {"-": r}})).replace("'-':","-"))

In [13]:
import tabulate
from ga4gh.vrs.normalize import normalize

# todo: this example should get changed to use normalized hgvs_g as input.
tlr.normalize = False

# Round-trip test: HGVS → VR Allele → HGVS[]
header = "check hgvs_orig sequence_id sequence_id_normalized hgvs_normalized".split()
table = [header]
for hgvs_expr in (
    "NC_000013.11:g.32936732_32936733del",
    "NC_000013.11:g.32936732_32936737del",
    "NC_000013.11:g.32936732_32936733insC",
    "NC_000013.11:g.32936732_32936733delinsC",
    "NC_000013.11:g.32936732_32936735delinsC",
    "NC_000013.11:g.32936732C>G",
    "NM_015102.3:n.2802C>T",
    "NC_000013.10:g.32331094_32331095dup",
    "NC_000013.10:g.32331092_32331093insTA"
):
    a = tlr.translate_from(hgvs_expr, "hgvs")
    he = tlr.translate_to(a, "hgvs")
    chk = "✔" if hgvs_expr in he else "✘"
    #print(f"{chk} {hgvs_expr}\n  → {ga4gh_identify(a)}\n  → {he}")
    a_norm = normalize(a, dp)
    row = [chk, hgvs_expr, ga4gh_identify(a), ga4gh_identify(a_norm), he[0] ]
    table += [row]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

check,hgvs_orig,sequence_id,sequence_id_normalized,hgvs_normalized
✔,NC_000013.11:g.32936732_32936733del,ga4gh:VA.yOoxi7-uUnJyn4QkQ23h6RJuT4Zqarow,ga4gh:VA.nEc6BuTmkSd14fDWNfviS6sJ7EwfYGDg,NC_000013.11:g.32936732_32936733del
✔,NC_000013.11:g.32936732_32936737del,ga4gh:VA.nJqbt_W7xV07irZ_F5mtsh5e5dkq9dBW,ga4gh:VA.WK3SzM4cqysOYUnfPmCknx37WEwQqq4K,NC_000013.11:g.32936732_32936737del
✘,NC_000013.11:g.32936732_32936733insC,ga4gh:VA.JEUN0DVx2gySgRhNDqlKYqehZxgKKlsY,ga4gh:VA.Ntfl2qZldiR4f-8piPhEjVcOfimtHVrF,NC_000013.11:g.32936733dup
✘,NC_000013.11:g.32936732_32936733delinsC,ga4gh:VA.cT0SNJb9bxB_KIhu2s6j37ZbTWaU4ozJ,ga4gh:VA.TEOEa--NI3gzjZn1uKMty73ZQe9YdW0R,NC_000013.11:g.32936733del
✘,NC_000013.11:g.32936732_32936735delinsC,ga4gh:VA.6ZgsF2lSBqMKcGL-xV-SUSrwN_UQTndJ,ga4gh:VA.9PjbePdxdFHuiuDvhH3h_u2b_o83-pL5,NC_000013.11:g.32936733_32936735del
✔,NC_000013.11:g.32936732C>G,ga4gh:VA.gvCtR5KLdng5G31DwajXiH6S3Gjhm5fh,ga4gh:VA.gvCtR5KLdng5G31DwajXiH6S3Gjhm5fh,NC_000013.11:g.32936732C>G
✔,NM_015102.3:n.2802C>T,ga4gh:VA.xa7CYW-FRyDcRuUW9EZ5-nye59t720Si,ga4gh:VA.xa7CYW-FRyDcRuUW9EZ5-nye59t720Si,NM_015102.3:n.2802C>T
✔,NC_000013.10:g.32331094_32331095dup,ga4gh:VA.gfK4gDYdDeLD_9n2KVSuIgD7IvqF-AW8,ga4gh:VA.3HTrI5MB9lwFTJuYJOzzkZPUWqHQs8u_,NC_000013.10:g.32331094_32331095dup
✘,NC_000013.10:g.32331092_32331093insTA,ga4gh:VA.z6Tv-sdFB5xKsw4-sC1DbL9Ps1Q45-Ja,ga4gh:VA.3HTrI5MB9lwFTJuYJOzzkZPUWqHQs8u_,NC_000013.10:g.32331094_32331095dup
